In [29]:
import requests
import pandas as pd
from lxml import etree
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import urllib3
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [30]:
class CustomError(Exception):
    pass

In [31]:
URL_DATA = 'https://sanctionslistservice.ofac.treas.gov/changes/latest'
NAMESPACE = {'ns': 'https://www.treasury.gov/ofac/DeltaFile/1.0'}

In [32]:
def fetch_data():
    '''Solicita el contenido XML desde el servidor'''
    try:
        response = requests.get(URL_DATA, verify=False)
        return response.content
    except requests.exceptions.RequestException as e:
        raise CustomError(f'Error al solicitar el contenido: {e}')
    
def parse_xml(content):
    '''Parsea el contenido XML y devuelve el elemento raiz'''
    return etree.fromstring(content)

In [33]:
def extract_publication_date(root):
    '''Extrae la fecha de publicación del XML'''
    date_element = root.find('ns:publicationInfo/ns:datePublished', NAMESPACE)
    return date_element.text.split("T")[0] if date_element is not None else "Fecha_no_disponible"

def is_only_number(text):
    '''Verifica si el texto contiene solo números'''
    return bool(re.fullmatch(r'\d+', text))

def extract_entity_data(entity):
    '''Extrae la informacion de una entiendad XML'''
    action = entity.get("action", "N/A")
    
    alias_text = []
    full_name_text = "N/A"

    for name in entity.findall(".//ns:name", NAMESPACE):
        alias_type = name.find("ns:aliasType", NAMESPACE)
        full_name = name.find(".//ns:translation[ns:script='Latin']/ns:formattedFullName", NAMESPACE)
        full_name = full_name.text if full_name is not None else "N/A"

        if alias_type != None:
            alias_text.append(full_name)
        else:
            if full_name_text == "N/A":
                full_name_text = full_name

    doc_text = []
    for doc in entity.findall("ns:identityDocuments/ns:identityDocument", NAMESPACE):
        if doc is not None:
            doc_type = doc.find("ns:type", NAMESPACE)
            doc_type_text = doc_type.text if doc_type is not None else "N/A"

            doc_number = doc.find("ns:documentNumber", NAMESPACE)
            doc_number_text = doc_number.text if doc_number is not None else "N/A"

            issuing_country = doc.find("ns:issuingCountry", NAMESPACE)
            issuing_country_text = issuing_country.text if issuing_country is not None else "N/A"

            if issuing_country_text == "Colombia":
                if doc_type_text == "Cedula No.":
                    doc_text.append(f"CC {doc_number_text}")
                elif doc_type_text == "NIT #":
                    doc_text.append(f"NIT {doc_number_text}")
            elif doc_type_text == "Passport":
                if is_only_number(doc_number_text):
                    doc_text.append(f"PAS {doc_number_text}")
                else:
                    doc_text.append(f"{doc_type_text} {doc_number_text}")
            else:
                doc_text.append(f"{doc_type_text} {doc_number_text}")
        else:
            doc_text.append("N/A")
    return {
        'Nombre Completo': full_name_text,
        'Documentos': doc_text,
        'Alias': alias_text,
        'Accion': action,
    }

In [34]:
def transform_data(content):
    '''Transforma el XML en un DataFrame, retorna el DataFrame y la fecha de publicación'''
    root = parse_xml(content)
    entities = root.findall('ns:entities/ns:entity', NAMESPACE)
    data = [extract_entity_data(entity) for entity in entities]
    return pd.DataFrame(data), extract_publication_date(root)

In [35]:
def save_to_excel(df, filename):
    '''Guarda el DataFrame en un archivo Excel y ajusta el ancho de las columnas'''
    df.to_excel(filename, index=False)
    wb = load_workbook(filename)
    ws = wb.active

    for col in ws.columns:
        max_length = 0
        col_letter = get_column_letter(col[0].column)

        for cell in col:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[col_letter].width = adjusted_width
        
    wb.save(filename)

In [36]:
def main():
    '''Funcion principal quen ejecuta el proceso completo'''
    try:
        content = fetch_data()
        df, pub_date = transform_data(content)

        filename = f'OFAC_{pub_date}.xlsx'
        save_to_excel(df, filename)
        print(f'Archivo guardado: {filename}')
    except CustomError as e:
        print(f'Error: {e}')

if __name__ == '__main__':
    main()

Archivo guardado: OFAC_2025-02-26.xlsx
